In [ ]:
#imports
import tensorflow as tf 
from tensorflow import keras 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn as sns
import PIL.Image as Image
import cv2
import os
import numpy as np
import pathlib

In [ ]:
#initialising dataset path
data_dir = '/kaggle/input/2000-hand-gestures/images' 
data_dir = pathlib.Path(data_dir)
data_dir

import os
#print(os.path.abspath("."))
#print(os.path.abspath(r".."))

dir_list = os.listdir("/kaggle/input/2000-hand-gestures/images")
print(dir_list)

In [ ]:
#listing classes
import glob
#closedFist = list(data_dir.glob('closed-fist/*'))[:600]
closedFist = list(data_dir.glob('closedFist/*'))[:600]
#print(closedFist)
fingerCircles = list(data_dir.glob('fingerCircle/*'))[:600]
fingerSymbols = list(data_dir.glob('fingerSymbols/*'))[:600]
multiFingerBend = list(data_dir.glob('multiFingerBend/*'))[:600]
openPalm = list(data_dir.glob('openPalm/*'))[:600]
semiOpenFist = list(data_dir.glob('semiOpenFist/*'))[:600]
semiOpenPalm = list(data_dir.glob('semiOpenPalm/*'))[:600]
singleFingerBend = list(data_dir.glob('singleFingerBend/*'))[:600]

In [ ]:
#showing sample of images 
fig, ax = plt.subplots(ncols=8, figsize=(20,5))
fig.suptitle('hand gesture classes')
closedFistImg = img.imread(closedFist[0])
fingerCirclesImg = img.imread(fingerCircles[0])
fingerSymbolsImg = img.imread(fingerSymbols[0])
multiFingerBendImg = img.imread(multiFingerBend[0])
openPalmImg = img.imread(openPalm[0])
semiOpenFistImg = img.imread(semiOpenFist[0])
semiOpenPalmImg = img.imread(semiOpenPalm[0])
singleFingerBendImg = img.imread(singleFingerBend[0])


ax[0].set_title('closed fist')
ax[1].set_title('finger circles')
ax[2].set_title('finger symbols')
ax[3].set_title('multi finger bend')
ax[4].set_title('open palm')
ax[5].set_title('semi open fist')
ax[6].set_title('semi open palm')
ax[7].set_title('single finger bend')

ax[0].imshow(closedFistImg)
ax[1].imshow(fingerCirclesImg)
ax[2].imshow(fingerSymbolsImg)
ax[3].imshow(multiFingerBendImg)
ax[4].imshow(openPalmImg)
ax[5].imshow(semiOpenFistImg)
ax[6].imshow(semiOpenPalmImg)
ax[7].imshow(singleFingerBendImg)

plt.show()

In [ ]:
df_images = {
    'open palm' : openPalm,
    'closed fist' : closedFist,
    'semi open palm' : semiOpenPalm,
    'semi open fist' : semiOpenFist,
    'finger circles': fingerCircles,
    'single finger bend' : singleFingerBend,
    'multi finger bend' : multiFingerBend,
    'finger symbols': fingerSymbols
}

df_labels = {
    'open palm' : 0,
    'closed fist' : 1,
    'semi open palm' : 2,
    'semi open fist' : 3,
    'finger circles': 4,
    'single finger bend' : 5,
    'multi finger bend' : 6,
    'finger symbols': 7
}

In [ ]:
img = cv2.imread(str(df_images['open palm'][0])) 
img.shape

In [ ]:
X, y = [], [] 
for label, images in df_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224)) 
        X.append(resized_img) 
        y.append(df_labels[label])
        
X = np.array(X)
X = X/255
y = np.array(y)

#spliting into test val & train
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val)

In [ ]:
#define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#fit the keras model on the dataset
model.fit(X_train, y_train, epochs=10, batch_size=10)

#evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
#show f1-score
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_bool))

In [ ]:
#plot model accuracy and loss
fig, ax = plt.subplots(ncols=2, figsize=(20,5))
#fig.suptitle('accuracy and loss')
ax[0].set_title('model accuracy')
ax[1].set_title('model loss')

ax[0].grid()
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('acc')
ax[0].plot(history.history['acc'], marker='8',color='red')
ax[0].plot(history.history['val_acc'], marker='8',color='green')
ax[0].legend(['train', 'val'])

ax[1].grid()
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('loss')
ax[1].plot(history.history['loss'], marker='8',color='red')
ax[1].plot(history.history['val_loss'], marker='8',color='green')
ax[1].legend(['train', 'val'])

plt.show()

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_bool)

plt.figure(figsize=(15,15))

ax = sns.heatmap(cm/np.sum(cm),fmt='.2%', annot=True, cmap='vlag')

ax.set_title('confusion matrix\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(df_labels)
ax.yaxis.set_ticklabels(df_labels)

plt.show()